In [1]:
import requests
import re
import numpy as np
import pandas as pd

## Gathering Data

In [2]:
# Get page and check if response if valid
page = requests.get("https://transmission.bpa.gov/business/operations/wind/baltwg.txt")
assert(page.status_code == 200)

In [3]:
# Encode raw bytes into utf-8 string
page_content = page.content.decode("utf-8")

## Data Extraction

In [4]:
# Seperate information header and numerical data
text = re.split('Date/Time', page_content)

# Information
print(text[0])

BPA Balancing Authority Load & Total Wind Generation
at 5-minute intervals, last 7 days
Dates: 29Aug2021 - 05Sep2021 (last updated 4Sep2021 21:01:03) Pacific Time
Based on 5-min MW readings from the BPA SCADA system for points 45583, 79687, 79682, 164377, 70681

This represents loads and resources in BPA's Balancing Authority (BA) including some that are not BPA's.
It does not include BPA loads served by transfer, scheduled out of region,
or scheduled to customers with their own BAs such as Seattle and Tacoma

BPA/Technical Operations (TOT-OpInfo@bpa.gov)




In [5]:
# Raw data
data = text[1]
data = '\nTimestamp\t' + data.lstrip()

# Sample data
data[0:100]

'\nTimestamp\tLoad\tWind\tHydro\tFossil/Biomass\tNuclear\r\n08/29/2021 00:00\t5125\t31\t5253\t1130\t1145\r\n08/29/20'

### Parse Data

In [6]:
# Parse rows
rows = str.split(data, sep = '\r')

# fix odd heading seperation
rows[0] = rows[0].replace('/','\t')

# Parse data seperated by tabs
matrix = [str.split(x,'\t') for x in rows]
df = pd.DataFrame([str.split(x,'\t') for x in rows])

# Clean leading white space
df[0]= df[0].str.lstrip()

# Setting header and index
header_row = 0
df.columns = df.iloc[header_row]
df = df.drop(header_row)
df = df.set_index("Timestamp")

### Working with missing values

In [7]:
# Remove empty column
df = df.drop(columns="Nuclear")

# Make missing values readable
df = df.replace("", np.NaN)

# Removing rows with less than 3 data points
df = df.dropna(how='all')

### Exporting clean data

In [8]:
df.to_csv("data/clean_energy_data.csv")